## TOC
* [Basic Expansion](#basic-expansion)
* [Expanding a policy](#expanding-iam-policies)
* [Expanding an obfuscated policy](#expanding-an-obfuscated-policy)
* [Fail if service does not exists](#fail-if-service-does-not-exists)

# Basic Expansion <a class="anchor" id="basic-expansion"></a>

In [1]:
from py_iam_expand.actions import expand_actions

action_string = "s3:Get*"
expanded_actions = expand_actions(action_string)
expanded_actions[:10]

['s3:GetAccelerateConfiguration',
 's3:GetAccessGrant',
 's3:GetAccessGrantsInstance',
 's3:GetAccessGrantsInstanceForPrefix',
 's3:GetAccessGrantsInstanceResourcePolicy',
 's3:GetAccessGrantsLocation',
 's3:GetAccessPoint',
 's3:GetAccessPointConfigurationForObjectLambda',
 's3:GetAccessPointForObjectLambda',
 's3:GetAccessPointPolicy']

## Expanding a policy <a class="anchor" id="expanding-iam-policies"></a>

In [2]:
import json
from py_iam_expand.policy import expand_policy_actions

policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "s3:GetB*",
                "Resource": "arn:aws:s3:::examplebucket/*"
            }
        ]
    }

expanded_policy = expand_policy_actions(policy_document)
print(json.dumps(expanded_policy, indent=2))

{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "s3:GetBucketAcl",
        "s3:GetBucketCORS",
        "s3:GetBucketLocation",
        "s3:GetBucketLogging",
        "s3:GetBucketMetadataTableConfiguration",
        "s3:GetBucketNotification",
        "s3:GetBucketObjectLockConfiguration",
        "s3:GetBucketOwnershipControls",
        "s3:GetBucketPolicy",
        "s3:GetBucketPolicyStatus",
        "s3:GetBucketPublicAccessBlock",
        "s3:GetBucketRequestPayment",
        "s3:GetBucketTagging",
        "s3:GetBucketVersioning",
        "s3:GetBucketWebsite"
      ],
      "Resource": "arn:aws:s3:::examplebucket/*"
    }
  ]
}


## Expanding an obfuscated policy <a class="anchor" id="expanding-an-obfuscated-policy"></a>

In this example, someone is trying to obfuscate the statements to avoid being detected by security scans.

In [3]:
import json
from py_iam_expand.policy import expand_policy_actions

policy_document = {
	"Version": "2012-10-17",
	"Statement": [
		{
			"Effect": "\u0041llow",
			"Action": "s3:Get..ce*",
			"Resource": "arn:aws:s3:::examplebucket/*"
		},
		{
			"Effect": "Deny",
			"NotAction": [
				" ec2:DescribeVo*",
				"iam:*AccessKey*"
			],
			"Resource": "*"
		}
	]
}

expanded_policy = expand_policy_actions(policy_document)
print(json.dumps(expanded_policy, indent=2))

{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [],
      "Resource": "arn:aws:s3:::examplebucket/*"
    },
    {
      "Effect": "Deny",
      "NotAction": [
        "ec2:DescribeVolumeAttribute",
        "ec2:DescribeVolumeStatus",
        "ec2:DescribeVolumes",
        "ec2:DescribeVolumesModifications",
        "iam:CreateAccessKey",
        "iam:DeleteAccessKey",
        "iam:GetAccessKeyLastUsed",
        "iam:ListAccessKeys",
        "iam:UpdateAccessKey"
      ],
      "Resource": "*"
    }
  ]
}


## Fail if service does not exists  <a class="anchor" id="fail-if-service-does-not-exists"></a>

In [4]:
import json
from py_iam_expand.policy import expand_policy_actions, InvalidActionHandling

policy_document = {
	"Version": "2012-10-17",
	"Statement": [
		{
			"Effect": "Allow",
			"Action": "s13:Get*",
			"Resource": "arn:aws:s3:::wrongservice/*"
		}
	]
}


try:
	expanded_policy = expand_policy_actions(policy_document, InvalidActionHandling.RAISE_ERROR)
	print(json.dumps(expanded_policy, indent=2))
except Exception as e:
	print(f"[ERROR]: {e}")

[ERROR]: Invalid action pattern 's13:Get*': Statement 0: Service 's13' not found
